#### Import libraries

In [1]:
import os 

import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

#### Read ERA-Land data as grid reference

In [2]:
ds_ref = xr.open_dataset(os.path.join(os.getcwd(), '..', '1.-Output_Gridded_data', 'valid_data_hourly_m_s_2020_2020.nc'))

#### Utility variables

In [3]:
key_columns = ['ALTITUD', 'LATITUD', 'LONGITUD', 'NOMBRE', 'NOM_PROV']

cuencas = list(range(0, 10))
cuencas.append('B')
cuencas.append('C')

#### Read Observation data

In [4]:
full_data = []

for cuenca in cuencas:
    print(cuenca)
    base_path = os.path.join(os.getcwd(), 'SfcWind_hourly_Spain_jan2019-sep2024', 'VientoHorario') 

    meta_obs_path = os.path.join(base_path, f'MaestroVientoHorario_Cuenca_{cuenca}.csv')
    data_obs_path = os.path.join(base_path, f'VientoHorario_Cuenca_{cuenca}.csv')

    meta_obs = pd.read_csv(meta_obs_path, delimiter=";", encoding="latin1", on_bad_lines="skip")
    data_obs = pd.read_csv(data_obs_path, delimiter=";", encoding="latin1", on_bad_lines="skip")

    #### Remove an extra spaec in INDICATIVO name column
    data_obs = data_obs.rename(columns={' INDICATIVO': 'INDICATIVO'})

    #### Mege metadata and data
    merged_df = pd.merge(meta_obs, data_obs, on='INDICATIVO', suffixes=('_meta', '_data'))

    #### Check if column information match
    for col in key_columns:
        col_df1 = f"{col}_meta"
        col_df2 = f"{col}_data"
        if not (merged_df[col_df1] == merged_df[col_df2]).all():
            mismatch = merged_df[merged_df[col_df1] != merged_df[col_df2]]
            print(f"Error: La columna {col} tiene discrepancias en las siguientes filas:")
            print(mismatch[[col_df1, col_df2]])
            raise ValueError(f"Discrepancias encontradas en la columna {col}")
    
    #### Remove columns duplicates
    merged_df = merged_df.loc[:, ~merged_df.columns.str.endswith('_meta')]
    merged_df.rename(columns={f'{col}_data': col for col in key_columns}, inplace=True)

    #### Change the order of the columns
    column_order = [
    'INDICATIVO', 'NOMBRE', 'NOM_PROV', 'NOM_MUNI', 'ALTITUD', 'LATITUD', 'LONGITUD',
    'C_X', 'C_Y', 'DATUM', 'IND_SYN', 'ALTA', 'BAJA', 'AÑO', 'MES', 'DIA', 'DIR_00', 
    'VEL_00', 'DIR_01', 'VEL_01', 'DIR_02', 'VEL_02', 'DIR_03', 
    'VEL_03', 'DIR_04', 'VEL_04', 'DIR_05', 'VEL_05', 'DIR_06', 'VEL_06', 'DIR_07', 
    'VEL_07', 'DIR_08', 'VEL_08', 'DIR_09', 'VEL_09', 'DIR_10', 'VEL_10', 'DIR_11', 
    'VEL_11', 'DIR_12', 'VEL_12', 'DIR_13', 'VEL_13', 'DIR_14', 'VEL_14', 'DIR_15', 
    'VEL_15', 'DIR_16', 'VEL_16', 'DIR_17', 'VEL_17', 'DIR_18', 'VEL_18', 'DIR_19', 
    'VEL_19', 'DIR_20', 'VEL_20', 'DIR_21', 'VEL_21', 'DIR_22', 'VEL_22', 'DIR_23', 
    'VEL_23', 'R_MAX_DIR', 'R_MAX_VEL', 'R_MAX_HOR', 'DIR_VEL_10', 'VEL_MED_10', 
    'HOR_VEL_10', 'REC24', 'REC77'
    ]    

    merged_df = merged_df[column_order]

    full_data.append(merged_df)

# Convert full_data (list of DataFrames) to a DataFrame
full_data = pd.concat(full_data, ignore_index=True)


0
1
2
3
4
5
6
7
8
9
B
C


In [6]:
full_data.columns

Index(['INDICATIVO', 'NOMBRE', 'NOM_PROV', 'NOM_MUNI', 'ALTITUD', 'LATITUD',
       'LONGITUD', 'C_X', 'C_Y', 'DATUM', 'IND_SYN', 'ALTA', 'BAJA', 'AÑO',
       'MES', 'DIA', 'DIR_00', 'VEL_00', 'DIR_01', 'VEL_01', 'DIR_02',
       'VEL_02', 'DIR_03', 'VEL_03', 'DIR_04', 'VEL_04', 'DIR_05', 'VEL_05',
       'DIR_06', 'VEL_06', 'DIR_07', 'VEL_07', 'DIR_08', 'VEL_08', 'DIR_09',
       'VEL_09', 'DIR_10', 'VEL_10', 'DIR_11', 'VEL_11', 'DIR_12', 'VEL_12',
       'DIR_13', 'VEL_13', 'DIR_14', 'VEL_14', 'DIR_15', 'VEL_15', 'DIR_16',
       'VEL_16', 'DIR_17', 'VEL_17', 'DIR_18', 'VEL_18', 'DIR_19', 'VEL_19',
       'DIR_20', 'VEL_20', 'DIR_21', 'VEL_21', 'DIR_22', 'VEL_22', 'DIR_23',
       'VEL_23', 'R_MAX_DIR', 'R_MAX_VEL', 'R_MAX_HOR', 'DIR_VEL_10',
       'VEL_MED_10', 'HOR_VEL_10', 'REC24', 'REC77'],
      dtype='object')